<a href="https://colab.research.google.com/github/samantamrityunjay/ICDcoding/blob/master/NonAI_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import re
import tqdm 
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
from ast import literal_eval
# import nltk
# nltk.download('punkt')
# from nltk.tokenize import word_tokenize
# import string
pd.set_option("display.max_colwidth",None)

In [3]:
from gensim.models import KeyedVectors
%time model = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/BioVecModels/biowordvec", binary=True,limit=1000000)

CPU times: user 14.6 s, sys: 1.24 s, total: 15.8 s
Wall time: 19.9 s


In [4]:
frequent_icd9category=['401','427','276','414','272','250','428','518','285','584']

In [5]:
diagnosis_icd9category_token=pd.read_csv("/content/drive/My Drive/AlgoIntern/Data/Data2.0/diagnosis_icd9category1.csv")

In [6]:
diagnosis_icd9category_token

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","['496', '553', '518', '453', '519', '276']","['hypercholesterolemia', 'cerebral', 'hypothyroidism', 'cough', 'accident', 'hypertension', 'artery', 'chest', 'vascular', 'coronary', 'fracture', 'injury', 'angina', 'pain', 'head']"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parking: main garage","['487', '416', '272', '530', '414', '553', '531', '518', '401', '519', '348', 'V02', '359', '584', '458', '578', '491', 'E93', '276', '244']","['pulmonary', 'hypertension', 'of', 'obstructive', 'discomfort', 'gastrointestinal', 'constipation', 'oxygen', 'breath', 'inflammation', 'artery', 'chest', 'hydrochlorothiazide', 'steroid', 'anxiety', 'omeprazole', 'senna', 'trazodone', 'failure', 'shortness', 'disease', 'pantoprazole', 'aspirin', 'hypothyroidism', 'zolpidem', 'furosemide', 'coronary', 

In [7]:
diagnosis_icd9category_token['medical_terms_list']=diagnosis_icd9category_token.medical_terms_list.apply(literal_eval)
diagnosis_icd9category_token['icd9_category']=diagnosis_icd9category_token.icd9_category.apply(literal_eval)

In [8]:
diagnosis_icd9category_token

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","[496, 553, 518, 453, 519, 276]","[hypercholesterolemia, cerebral, hypothyroidism, cough, accident, hypertension, artery, chest, vascular, coronary, fracture, injury, angina, pain, head]"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parking: main garage","[487, 416, 272, 530, 414, 553, 531, 518, 401, 519, 348, V02, 359, 584, 458, 578, 491, E93, 276, 244]","[pulmonary, hypertension, of, obstructive, discomfort, gastrointestinal, constipation, oxygen, breath, inflammation, artery, chest, hydrochlorothiazide, steroid, anxiety, omeprazole, senna, trazodone, failure, shortness, disease, pantoprazole, aspirin, hypothyroidism, zolpidem, furosemide, coronary, prednisone, lethargy, steroids, ulcer, chronic, respiratory, morphine, pain]"
2,135453.0,discharge diagnosis: 1. cervical spondylosis wi

In [9]:
category_med_terms={}
common_diagnosis_terms=["monday","telephone","of","phone"]
for i in frequent_icd9category:
    diagnosis_x=diagnosis_icd9category_token.loc[diagnosis_icd9category_token.icd9_category.apply(lambda x: i in x)].copy()
    y=diagnosis_x.medical_terms_list.to_list()    
    z=[list(set(a)) for a in y]
    z1=[a for b in z for a in b]
    dict_med=dict(Counter(z1))
    dict_med=dict(sorted(dict_med.items(),key= lambda x: x[1],reverse=True))
    dict_med_list=list(dict_med)
    dict_med_list1=[a for a in dict_med_list if a not in common_diagnosis_terms]
    category_med_terms[i]=dict_med_list1[:50]

diagnosis_icdcategory_medterms={'icd9category':[i for i in category_med_terms],
                                'frequent_medterms':[category_med_terms[i] for i in category_med_terms]}
diagnosis_icdcategory_medterms_df=pd.DataFrame.from_dict(diagnosis_icdcategory_medterms)

diagnosis_icdcategory_medterms_df

,icd9category,frequent_medterms
0,401,"[pain, fever, chest, shortness, infection, breath, redness, aspirin, hypertension, atrial, fibrillation, chills, disease, coumadin, bleeding, failure, pneumonia, vomiting, fevers, constipation, swelling, nausea, abdominal, heart, diabetes, weakness, cough, anemia, erythema, headache, metoprolol, confusion, numbness, mouth, weight, diarrhea, gain, loss, lisinopril, blood, cancer, seizures, pulmonary, infarction, myocardial, bleed, stroke, motrin, artery, tylenol]"
1,427,"[pain, atrial, fibrillation, chest, shortness, fever, coumadin, infection, breath, failure, disease, heart, redness, pneumonia, aspirin, chills, hypertension, bleeding, metoprolol, fevers, renal, vomiting, anemia, abdominal, nausea, erythema, sodium, diabetes, chronic, constipation, weight, pulmonary, gain, cough, swelling, mouth, cancer, warfarin, diarrhea, lisinopril, bleed, artery, infarction, stroke, myocardial, blood, urinary, respiratory, oxygen, congestive]"
2,276,"[pain, shortness, chest, infection, breath, failure, fever, pneumonia, disease, chills, atrial, fevers, fibrillation, vomiting, bleeding, nausea, abdominal, renal, hypertension, heart, aspirin, anemia, coumadin, diarrhea, diabetes, metoprolol, mouth, chronic, constipation, cough, redness, respiratory, swelling, sodium, urinary, confusion, blood, cancer, acute, pulmonary, oxygen, tract, alcohol, lisinopril, loss, bleed, kidney, dehydration, weakness, sepsis]"
3,414,"[pain, chest, fever, shortness, infection, breath, disease, redness, atrial, failure, fibrillation, heart, aspirin, coumadin, hypertension, myocardial, infarction, erythema, bleeding, weight, gain, diabetes, chills, metoprolol, artery, renal, coronary, pneumonia, fevers, lisinopril, sodium, anemia, chronic, vomiting, mellitus, nausea, abdominal, pulmonary, constipation, edema, swelling, congestive, bleed, cancer, cholesterol, cough, mouth, blood, loss, hyperlipidemia]"
4,272,"[pain, fever, chest, infection, shortness, breath, redness, aspirin, atrial, disease, fibrillation, failure, coumadin, hypertension, bleeding, chills, heart, erythema, pneumonia, diabetes, constipation, vomiting, fevers, metoprolol, swelling, weight, renal, gain, nausea, abdominal, anemia, infarction, myocardial, cough, lisinopril, mouth, weakness, cancer, artery, confusion, mellitus, headache, pulmonary, loss, chronic, diarrhea, hyperlipidemia, blood, numbness, tylenol]"
5,250,"[pain, chest, shortness, breath, diabetes, fever, infection, failure, disease, chills, atrial, hypertension, heart, fibrillation, aspirin, fevers, mellitus, redness, pneumonia, bleeding, vomiting, renal, coumadin, nausea, abdominal, anemia, metoprolol, chronic, constipation, swelling, sodium, blood, cough, mouth, lisinopril, diarrhea, confusion, artery, erythema, pulmonary, urinary, respiratory, infarction, loss, myocardial, bleed, coronary, weakness, weight, oxygen]"
6,428,"[pain, failure, shortness, chest, heart, breath, atrial, fibrillation, disease, infection, fever, pneumonia, coumadin, chills, renal, hypertension, sodium, fevers, aspirin, metoprolol, bleeding, chronic, anemia, diabetes, congestive, redness, pulmonary, vomiting, nausea, lisinopril, abdominal, oxygen, myocardial, infarction, respiratory, artery, systolic, coronary, mouth, cough, urinary, blood, mellitus, swelling, tract, weight, bleed, diarrhea, acute, gain]"
7,518,"[pain, shortness, breath, pneumonia, failure, chest, fever, infection, disease, chills, respiratory, atrial, fibrillation, fevers, vomiting, heart, hypertension, cough, abdominal, nausea, renal, coumadin, anemia, pulmonary, redness, bleeding, aspirin, oxygen, swelling, constipation, diarrhea, mouth, chronic, cancer, diabetes, metoprolol, sodium, confusion, fracture, blood, acute, sepsis, urinary, loss, weakness, lung, prednisone, tract, obstructive, edema]"
8,285,"[pain, shortness, chest, breath, infection, fever, disease, anemia, bleeding, failure, chills, pneumonia, fevers, atrial, fibrillation, aspirin, blood, co

In [10]:
y=diagnosis_icdcategory_medterms_df.frequent_medterms.to_list()
y1=[a for b in y for a in b]
medical=dict(Counter(y1))
print([k for (k,v) in medical.items() if v>=16])

[]


In [11]:
bagofwords=list(set([a for b in diagnosis_icdcategory_medterms_df.frequent_medterms.to_list() for a in b]))

In [12]:
len(bagofwords)

76

In [13]:
set(bagofwords).issubset(model.vocab)

True

In [14]:
simi_matrix=np.zeros((len(bagofwords),len(bagofwords)))

In [15]:
simi_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
simi_matrix.shape

(76, 76)

In [17]:
for i in tqdm(range(len(bagofwords))):
  for j in range(len(bagofwords)):
    simi_matrix[i][j]=model.similarity(bagofwords[i],bagofwords[j])

100%|██████████| 76/76 [00:00<00:00, 587.17it/s]


In [18]:
simi_matrix

array([[1.        , 0.70788604, 0.43498385, ..., 0.44902244, 0.28360549,
        0.74148297],
       [0.70788604, 1.        , 0.52722788, ..., 0.45991182, 0.32771045,
        0.67368197],
       [0.43498385, 0.52722788, 1.        , ..., 0.50694972, 0.49392176,
        0.45901039],
       ...,
       [0.44902244, 0.45991182, 0.50694972, ..., 1.        , 0.36671904,
        0.38548869],
       [0.28360549, 0.32771045, 0.49392176, ..., 0.36671904, 1.        ,
        0.31812355],
       [0.74148297, 0.67368197, 0.45901039, ..., 0.38548869, 0.31812355,
        1.        ]])

In [19]:
features={}
for i in range(len(bagofwords)-1):
  similar=[]
  for j in range(i+1,len(bagofwords)):
    if simi_matrix[i][j]>=0.7:
      similar.append(bagofwords[j])
  features[bagofwords[i]]=similar
features[bagofwords[-1]]=[]    


In [20]:
print(features)

{'chills': ['vomiting', 'dizziness', 'shortness', 'nausea', 'fevers', 'fever'], 'vomiting': ['diarrhea', 'dizziness', 'nausea', 'constipation'], 'dehydration': [], 'seizures': [], 'edema': [], 'breath': [], 'confusion': [], 'blood': [], 'abdominal': [], 'diarrhea': ['nausea', 'constipation'], 'kidney': ['renal'], 'chronic': ['acute'], 'cough': [], 'dizziness': ['headache', 'nausea', 'numbness'], 'gain': [], 'shortness': ['fevers'], 'warfarin': ['aspirin'], 'headache': ['nausea', 'pain'], 'redness': ['erythema'], 'atrial': ['fibrillation'], 'cancer': [], 'chest': [], 'hyperlipidemia': ['mellitus', 'hypertension'], 'erythema': [], 'disease': [], 'pulmonary': ['lung'], 'tract': [], 'heart': [], 'pneumonia': ['sepsis'], 'weight': [], 'coronary': ['myocardial', 'artery'], 'myocardial': ['infarction'], 'mellitus': ['diabetes'], 'systolic': [], 'stroke': ['infarction'], 'hypertension': [], 'swelling': [], 'nausea': ['constipation'], 'weakness': ['numbness'], 'mouth': [], 'fevers': ['fever'], 

In [21]:
len(features)

76

In [22]:
features_list=[features[k]+[k] for k in list(features)]

In [23]:
features_list

[['vomiting', 'dizziness', 'shortness', 'nausea', 'fevers', 'fever', 'chills'],
 ['diarrhea', 'dizziness', 'nausea', 'constipation', 'vomiting'],
 ['dehydration'],
 ['seizures'],
 ['edema'],
 ['breath'],
 ['confusion'],
 ['blood'],
 ['abdominal'],
 ['nausea', 'constipation', 'diarrhea'],
 ['renal', 'kidney'],
 ['acute', 'chronic'],
 ['cough'],
 ['headache', 'nausea', 'numbness', 'dizziness'],
 ['gain'],
 ['fevers', 'shortness'],
 ['aspirin', 'warfarin'],
 ['nausea', 'pain', 'headache'],
 ['erythema', 'redness'],
 ['fibrillation', 'atrial'],
 ['cancer'],
 ['chest'],
 ['mellitus', 'hypertension', 'hyperlipidemia'],
 ['erythema'],
 ['disease'],
 ['lung', 'pulmonary'],
 ['tract'],
 ['heart'],
 ['sepsis', 'pneumonia'],
 ['weight'],
 ['myocardial', 'artery', 'coronary'],
 ['infarction', 'myocardial'],
 ['diabetes', 'mellitus'],
 ['systolic'],
 ['infarction', 'stroke'],
 ['hypertension'],
 ['swelling'],
 ['constipation', 'nausea'],
 ['numbness', 'weakness'],
 ['mouth'],
 ['fever', 'fevers'],


In [24]:
len(features_list)

76

In [25]:
#Python code to merge all sublist having common elements. 

#Importing 
from collections import defaultdict 

#merge function to merge all sublist having common elements. 
def merge_common(lists): 
	neigh = defaultdict(set) 
	visited = set() 
	for each in lists: 
		for item in each: 
			neigh[item].update(each) 
	def comp(node, neigh = neigh, visited = visited, vis = visited.add): 
		nodes = set([node]) 
		next_node = nodes.pop 
		while nodes: 
			node = next_node() 
			vis(node) 
			nodes |= neigh[node] - visited 
			yield node 
	for node in neigh: 
		if node not in visited: 
			yield sorted(comp(node)) 

# #Input list initialization 
# Input = [['z','x','y'], ['y','g','e'], ['z'],['x','p'], 
# 		['a','b'], ['y','a'], ['d','g']] 

#Calling merge function 
condensed_feature = list(merge_common(features_list)) 

#Printing function 
# print("Initial list of list is :") 
# print(Input) 

# print("List of list after merging is:") 
# print(Output) 


In [26]:
condensed_feature

[['chills',
  'constipation',
  'diarrhea',
  'dizziness',
  'fever',
  'fevers',
  'headache',
  'nausea',
  'numbness',
  'pain',
  'shortness',
  'vomiting',
  'weakness'],
 ['dehydration'],
 ['seizures'],
 ['edema'],
 ['breath'],
 ['confusion'],
 ['blood'],
 ['abdominal'],
 ['kidney', 'renal'],
 ['acute', 'chronic'],
 ['cough'],
 ['gain'],
 ['aspirin', 'warfarin'],
 ['erythema', 'redness'],
 ['atrial', 'fibrillation'],
 ['cancer'],
 ['chest'],
 ['diabetes', 'hyperlipidemia', 'hypertension', 'mellitus'],
 ['disease'],
 ['lung', 'pulmonary'],
 ['tract'],
 ['heart'],
 ['pneumonia', 'sepsis'],
 ['weight'],
 ['artery', 'coronary', 'infarction', 'myocardial', 'stroke'],
 ['systolic'],
 ['swelling'],
 ['mouth'],
 ['urinary'],
 ['lisinopril', 'metoprolol'],
 ['respiratory'],
 ['oxygen'],
 ['motrin', 'tylenol'],
 ['prednisone'],
 ['obstructive'],
 ['gastrointestinal'],
 ['fracture'],
 ['congestive', 'failure'],
 ['anemia'],
 ['bleed', 'bleeding'],
 ['alcohol'],
 ['coumadin'],
 ['cholesterol

In [27]:
len(condensed_feature)

46

In [28]:
model.similarity('abdominal','edema')

0.40815303

In [29]:
model.similarity('gain','loss')

0.6596458

In [30]:
condensed_feature_dict={}
count=0
for i in condensed_feature:
  condensed_feature_dict['f'+str(count)]=i
  count+=1

In [31]:
print(condensed_feature_dict)

{'f0': ['chills', 'constipation', 'diarrhea', 'dizziness', 'fever', 'fevers', 'headache', 'nausea', 'numbness', 'pain', 'shortness', 'vomiting', 'weakness'], 'f1': ['dehydration'], 'f2': ['seizures'], 'f3': ['edema'], 'f4': ['breath'], 'f5': ['confusion'], 'f6': ['blood'], 'f7': ['abdominal'], 'f8': ['kidney', 'renal'], 'f9': ['acute', 'chronic'], 'f10': ['cough'], 'f11': ['gain'], 'f12': ['aspirin', 'warfarin'], 'f13': ['erythema', 'redness'], 'f14': ['atrial', 'fibrillation'], 'f15': ['cancer'], 'f16': ['chest'], 'f17': ['diabetes', 'hyperlipidemia', 'hypertension', 'mellitus'], 'f18': ['disease'], 'f19': ['lung', 'pulmonary'], 'f20': ['tract'], 'f21': ['heart'], 'f22': ['pneumonia', 'sepsis'], 'f23': ['weight'], 'f24': ['artery', 'coronary', 'infarction', 'myocardial', 'stroke'], 'f25': ['systolic'], 'f26': ['swelling'], 'f27': ['mouth'], 'f28': ['urinary'], 'f29': ['lisinopril', 'metoprolol'], 'f30': ['respiratory'], 'f31': ['oxygen'], 'f32': ['motrin', 'tylenol'], 'f33': ['prednis

In [32]:
diagnosis_icd9category_token

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","[496, 553, 518, 453, 519, 276]","[hypercholesterolemia, cerebral, hypothyroidism, cough, accident, hypertension, artery, chest, vascular, coronary, fracture, injury, angina, pain, head]"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parking: main garage","[487, 416, 272, 530, 414, 553, 531, 518, 401, 519, 348, V02, 359, 584, 458, 578, 491, E93, 276, 244]","[pulmonary, hypertension, of, obstructive, discomfort, gastrointestinal, constipation, oxygen, breath, inflammation, artery, chest, hydrochlorothiazide, steroid, anxiety, omeprazole, senna, trazodone, failure, shortness, disease, pantoprazole, aspirin, hypothyroidism, zolpidem, furosemide, coronary, prednisone, lethargy, steroids, ulcer, chronic, respiratory, morphine, pain]"
2,135453.0,discharge diagnosis: 1. cervical spondylosis wi

In [33]:
condensed_feature

[['chills',
  'constipation',
  'diarrhea',
  'dizziness',
  'fever',
  'fevers',
  'headache',
  'nausea',
  'numbness',
  'pain',
  'shortness',
  'vomiting',
  'weakness'],
 ['dehydration'],
 ['seizures'],
 ['edema'],
 ['breath'],
 ['confusion'],
 ['blood'],
 ['abdominal'],
 ['kidney', 'renal'],
 ['acute', 'chronic'],
 ['cough'],
 ['gain'],
 ['aspirin', 'warfarin'],
 ['erythema', 'redness'],
 ['atrial', 'fibrillation'],
 ['cancer'],
 ['chest'],
 ['diabetes', 'hyperlipidemia', 'hypertension', 'mellitus'],
 ['disease'],
 ['lung', 'pulmonary'],
 ['tract'],
 ['heart'],
 ['pneumonia', 'sepsis'],
 ['weight'],
 ['artery', 'coronary', 'infarction', 'myocardial', 'stroke'],
 ['systolic'],
 ['swelling'],
 ['mouth'],
 ['urinary'],
 ['lisinopril', 'metoprolol'],
 ['respiratory'],
 ['oxygen'],
 ['motrin', 'tylenol'],
 ['prednisone'],
 ['obstructive'],
 ['gastrointestinal'],
 ['fracture'],
 ['congestive', 'failure'],
 ['anemia'],
 ['bleed', 'bleeding'],
 ['alcohol'],
 ['coumadin'],
 ['cholesterol

In [34]:
all_vocab=[a for b in condensed_feature for a in b]

In [35]:
all_vocab

['chills',
 'constipation',
 'diarrhea',
 'dizziness',
 'fever',
 'fevers',
 'headache',
 'nausea',
 'numbness',
 'pain',
 'shortness',
 'vomiting',
 'weakness',
 'dehydration',
 'seizures',
 'edema',
 'breath',
 'confusion',
 'blood',
 'abdominal',
 'kidney',
 'renal',
 'acute',
 'chronic',
 'cough',
 'gain',
 'aspirin',
 'warfarin',
 'erythema',
 'redness',
 'atrial',
 'fibrillation',
 'cancer',
 'chest',
 'diabetes',
 'hyperlipidemia',
 'hypertension',
 'mellitus',
 'disease',
 'lung',
 'pulmonary',
 'tract',
 'heart',
 'pneumonia',
 'sepsis',
 'weight',
 'artery',
 'coronary',
 'infarction',
 'myocardial',
 'stroke',
 'systolic',
 'swelling',
 'mouth',
 'urinary',
 'lisinopril',
 'metoprolol',
 'respiratory',
 'oxygen',
 'motrin',
 'tylenol',
 'prednisone',
 'obstructive',
 'gastrointestinal',
 'fracture',
 'congestive',
 'failure',
 'anemia',
 'bleed',
 'bleeding',
 'alcohol',
 'coumadin',
 'cholesterol',
 'infection',
 'loss',
 'sodium']

In [36]:
diagnosis_icd9category_token['med_terms_condensed']=diagnosis_icd9category_token.medical_terms_list.progress_apply(lambda x:list(set(x).intersection(set(all_vocab))))

In [37]:
diagnosis_icd9category_token

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list,med_terms_condensed
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","[496, 553, 518, 453, 519, 276]","[hypercholesterolemia, cerebral, hypothyroidism, cough, accident, hypertension, artery, chest, vascular, coronary, fracture, injury, angina, pain, head]","[hypertension, fracture, artery, chest, pain, coronary, cough]"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parking: main garage","[487, 416, 272, 530, 414, 553, 531, 518, 401, 519, 348, V02, 359, 584, 458, 578, 491, E93, 276, 244]","[pulmonary, hypertension, of, obstructive, discomfort, gastrointestinal, constipation, oxygen, breath, inflammation, artery, chest, hydrochlorothiazide, steroid, anxiety, omeprazole, senna, trazodone, failure, shortness, disease, pantoprazole, aspirin, hypothyroidism, zolpidem, furosemide, coronary, prednisone, lethargy, steroids, ulcer, chronic, res

In [38]:
def return_feature(x):
  feature=[]
  for i in x:
    for j in list(condensed_feature_dict):
      if i in condensed_feature_dict[j]:
        feature.append(j)
  return list(set(feature))


In [39]:
diagnosis_icd9category_token['features']=diagnosis_icd9category_token.med_terms_condensed.progress_apply(lambda x:return_feature(x))

In [40]:
diagnosis_icd9category_token

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list,med_terms_condensed,features
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","[496, 553, 518, 453, 519, 276]","[hypercholesterolemia, cerebral, hypothyroidism, cough, accident, hypertension, artery, chest, vascular, coronary, fracture, injury, angina, pain, head]","[hypertension, fracture, artery, chest, pain, coronary, cough]","[f0, f10, f36, f17, f16, f24]"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parking: main garage","[487, 416, 272, 530, 414, 553, 531, 518, 401, 519, 348, V02, 359, 584, 458, 578, 491, E93, 276, 244]","[pulmonary, hypertension, of, obstructive, discomfort, gastrointestinal, constipation, oxygen, breath, inflammation, artery, chest, hydrochlorothiazide, steroid, anxiety, omeprazole, senna, trazodone, failure, shortness, disease, pantoprazole, aspirin, hypothyroidism, zolpidem, furosemide, coronary, prednisone

In [41]:
diagnosis_icdcategory_medterms_df

,icd9category,frequent_medterms
0,401,"[pain, fever, chest, shortness, infection, breath, redness, aspirin, hypertension, atrial, fibrillation, chills, disease, coumadin, bleeding, failure, pneumonia, vomiting, fevers, constipation, swelling, nausea, abdominal, heart, diabetes, weakness, cough, anemia, erythema, headache, metoprolol, confusion, numbness, mouth, weight, diarrhea, gain, loss, lisinopril, blood, cancer, seizures, pulmonary, infarction, myocardial, bleed, stroke, motrin, artery, tylenol]"
1,427,"[pain, atrial, fibrillation, chest, shortness, fever, coumadin, infection, breath, failure, disease, heart, redness, pneumonia, aspirin, chills, hypertension, bleeding, metoprolol, fevers, renal, vomiting, anemia, abdominal, nausea, erythema, sodium, diabetes, chronic, constipation, weight, pulmonary, gain, cough, swelling, mouth, cancer, warfarin, diarrhea, lisinopril, bleed, artery, infarction, stroke, myocardial, blood, urinary, respiratory, oxygen, congestive]"
2,276,"[pain, shortness, chest, infection, breath, failure, fever, pneumonia, disease, chills, atrial, fevers, fibrillation, vomiting, bleeding, nausea, abdominal, renal, hypertension, heart, aspirin, anemia, coumadin, diarrhea, diabetes, metoprolol, mouth, chronic, constipation, cough, redness, respiratory, swelling, sodium, urinary, confusion, blood, cancer, acute, pulmonary, oxygen, tract, alcohol, lisinopril, loss, bleed, kidney, dehydration, weakness, sepsis]"
3,414,"[pain, chest, fever, shortness, infection, breath, disease, redness, atrial, failure, fibrillation, heart, aspirin, coumadin, hypertension, myocardial, infarction, erythema, bleeding, weight, gain, diabetes, chills, metoprolol, artery, renal, coronary, pneumonia, fevers, lisinopril, sodium, anemia, chronic, vomiting, mellitus, nausea, abdominal, pulmonary, constipation, edema, swelling, congestive, bleed, cancer, cholesterol, cough, mouth, blood, loss, hyperlipidemia]"
4,272,"[pain, fever, chest, infection, shortness, breath, redness, aspirin, atrial, disease, fibrillation, failure, coumadin, hypertension, bleeding, chills, heart, erythema, pneumonia, diabetes, constipation, vomiting, fevers, metoprolol, swelling, weight, renal, gain, nausea, abdominal, anemia, infarction, myocardial, cough, lisinopril, mouth, weakness, cancer, artery, confusion, mellitus, headache, pulmonary, loss, chronic, diarrhea, hyperlipidemia, blood, numbness, tylenol]"
5,250,"[pain, chest, shortness, breath, diabetes, fever, infection, failure, disease, chills, atrial, hypertension, heart, fibrillation, aspirin, fevers, mellitus, redness, pneumonia, bleeding, vomiting, renal, coumadin, nausea, abdominal, anemia, metoprolol, chronic, constipation, swelling, sodium, blood, cough, mouth, lisinopril, diarrhea, confusion, artery, erythema, pulmonary, urinary, respiratory, infarction, loss, myocardial, bleed, coronary, weakness, weight, oxygen]"
6,428,"[pain, failure, shortness, chest, heart, breath, atrial, fibrillation, disease, infection, fever, pneumonia, coumadin, chills, renal, hypertension, sodium, fevers, aspirin, metoprolol, bleeding, chronic, anemia, diabetes, congestive, redness, pulmonary, vomiting, nausea, lisinopril, abdominal, oxygen, myocardial, infarction, respiratory, artery, systolic, coronary, mouth, cough, urinary, blood, mellitus, swelling, tract, weight, bleed, diarrhea, acute, gain]"
7,518,"[pain, shortness, breath, pneumonia, failure, chest, fever, infection, disease, chills, respiratory, atrial, fibrillation, fevers, vomiting, heart, hypertension, cough, abdominal, nausea, renal, coumadin, anemia, pulmonary, redness, bleeding, aspirin, oxygen, swelling, constipation, diarrhea, mouth, chronic, cancer, diabetes, metoprolol, sodium, confusion, fracture, blood, acute, sepsis, urinary, loss, weakness, lung, prednisone, tract, obstructive, edema]"
8,285,"[pain, shortness, chest, breath, infection, fever, disease, anemia, bleeding, failure, chills, pneumonia, fevers, atrial, fibrillation, aspirin, blood, co

In [42]:
diagnosis_icdcategory_medterms_df['feature']=diagnosis_icdcategory_medterms_df.frequent_medterms.progress_apply(lambda x:return_feature(x))

In [43]:
diagnosis_icdcategory_medterms_df

,icd9category,frequent_medterms,feature
0,401,"[pain, fever, chest, shortness, infection, breath, redness, aspirin, hypertension, atrial, fibrillation, chills, disease, coumadin, bleeding, failure, pneumonia, vomiting, fevers, constipation, swelling, nausea, abdominal, heart, diabetes, weakness, cough, anemia, erythema, headache, metoprolol, confusion, numbness, mouth, weight, diarrhea, gain, loss, lisinopril, blood, cancer, seizures, pulmonary, infarction, myocardial, bleed, stroke, motrin, artery, tylenol]","[f29, f4, f37, f24, f7, f0, f44, f22, f32, f18, f15, f27, f23, f43, f10, f2, f38, f17, f13, f5, f16, f19, f39, f26, f41, f21, f6, f12, f14, f11]"
1,427,"[pain, atrial, fibrillation, chest, shortness, fever, coumadin, infection, breath, failure, disease, heart, redness, pneumonia, aspirin, chills, hypertension, bleeding, metoprolol, fevers, renal, vomiting, anemia, abdominal, nausea, erythema, sodium, diabetes, chronic, constipation, weight, pulmonary, gain, cough, swelling, mouth, cancer, warfarin, diarrhea, lisinopril, bleed, artery, infarction, stroke, myocardial, blood, urinary, respiratory, oxygen, congestive]","[f11, f29, f4, f45, f37, f24, f7, f0, f22, f31, f9, f18, f15, f27, f23, f28, f43, f10, f38, f17, f13, f16, f19, f30, f39, f26, f41, f21, f6, f12, f14, f8]"
2,276,"[pain, shortness, chest, infection, breath, failure, fever, pneumonia, disease, chills, atrial, fevers, fibrillation, vomiting, bleeding, nausea, abdominal, renal, hypertension, heart, aspirin, anemia, coumadin, diarrhea, diabetes, metoprolol, mouth, chronic, constipation, cough, redness, respiratory, swelling, sodium, urinary, confusion, blood, cancer, acute, pulmonary, oxygen, tract, alcohol, lisinopril, loss, bleed, kidney, dehydration, weakness, sepsis]","[f29, f4, f1, f37, f45, f7, f0, f44, f22, f40, f31, f9, f18, f20, f15, f27, f28, f43, f10, f38, f17, f13, f5, f16, f19, f30, f39, f26, f41, f21, f6, f12, f14, f8]"
3,414,"[pain, chest, fever, shortness, infection, breath, disease, redness, atrial, failure, fibrillation, heart, aspirin, coumadin, hypertension, myocardial, infarction, erythema, bleeding, weight, gain, diabetes, chills, metoprolol, artery, renal, coronary, pneumonia, fevers, lisinopril, sodium, anemia, chronic, vomiting, mellitus, nausea, abdominal, pulmonary, constipation, edema, swelling, congestive, bleed, cancer, cholesterol, cough, mouth, blood, loss, hyperlipidemia]","[f11, f29, f4, f45, f37, f24, f7, f0, f44, f22, f9, f3, f18, f15, f42, f27, f23, f43, f10, f38, f17, f13, f16, f19, f39, f26, f41, f21, f6, f12, f14, f8]"
4,272,"[pain, fever, chest, infection, shortness, breath, redness, aspirin, atrial, disease, fibrillation, failure, coumadin, hypertension, bleeding, chills, heart, erythema, pneumonia, diabetes, constipation, vomiting, fevers, metoprolol, swelling, weight, renal, gain, nausea, abdominal, anemia, infarction, myocardial, cough, lisinopril, mouth, weakness, cancer, artery, confusion, mellitus, headache, pulmonary, loss, chronic, diarrhea, hyperlipidemia, blood, numbness, tylenol]","[f29, f4, f37, f24, f7, f0, f44, f22, f32, f9, f18, f15, f27, f23, f43, f10, f38, f17, f13, f5, f16, f8, f19, f39, f26, f41, f21, f6, f12, f14, f11]"
5,250,"[pain, chest, shortness, breath, diabetes, fever, infection, failure, disease, chills, atrial, hypertension, heart, fibrillation, aspirin, fevers, mellitus, redness, pneumonia, bleeding, vomiting, renal, coumadin, nausea, abdominal, anemia, metoprolol, chronic, constipation, swelling, sodium, blood, cough, mouth, lisinopril, diarrhea, confusion, artery, erythema, pulmonary, urinary, respiratory, infarction, loss, myocardial, bleed, coronary, weakness, weight, oxygen]","[f29, f4, f45, f37, f24, f7, f0, f44, f22, f31, f9, f18, f27, f23, f28, f43, f10, f38, f17, f13, f5, f16, f19, f30, f39, f26, f41, f21, f6, f12, f14, f8]"
6,428,"[pain, failure, shortness, chest, heart, breath, atrial, fibrillation, disease, infection, fever, pneumonia, coumadin, chills, renal, hypertension, sodium

In [44]:
def predict_icd(x):
  category_weights={}
  if len(x):
    for item1 in frequent_icd9category:
      category_features=[a for b in diagnosis_icdcategory_medterms_df.loc[diagnosis_icdcategory_medterms_df.icd9category==item1].feature.to_list() for a in b]
      count=len(list(set(x).intersection(set(category_features))))
      category_weights[item1]=count/len(x)/len(category_features)
  else:
    for item1 in frequent_icd9category:
      category_weights[item1]=0.0

      
  return category_weights
  

In [45]:
diagnosis_icd9category_token['predicted_icd_weigths']=diagnosis_icd9category_token.features.progress_apply(lambda x:predict_icd(x))

In [46]:
diagnosis_icd9category_token

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list,med_terms_condensed,features,predicted_icd_weigths
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","[496, 553, 518, 453, 519, 276]","[hypercholesterolemia, cerebral, hypothyroidism, cough, accident, hypertension, artery, chest, vascular, coronary, fracture, injury, angina, pain, head]","[hypertension, fracture, artery, chest, pain, coronary, cough]","[f0, f10, f36, f17, f16, f24]","{'401': 0.02777777777777778, '427': 0.026041666666666668, '276': 0.0196078431372549, '414': 0.026041666666666668, '272': 0.026881720430107527, '250': 0.026041666666666668, '428': 0.025252525252525252, '518': 0.02314814814814815, '285': 0.02380952380952381, '584': 0.020833333333333332}"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parking: main garage","[487, 416, 272, 530, 414, 553, 531, 518, 401, 519, 348, V02, 359, 584, 458, 578, 491, E93, 276, 244]","[

In [47]:
def predictbasedonwts(x):
  x1=dict(sorted(x.items(),key= lambda y: y[1],reverse=True))
  x2={k:v for (k,v) in x1.items() if v>0}
  repeatedval=dict(Counter(list(x2.values())))
  repeatedvalgreaterthan1=[k for (k,v) in repeatedval.items() if v>1]
  x3=dict(list(x2.items())[:10])
  x4=[v for (k,v) in x3.items() if v in repeatedvalgreaterthan1]
  x5={k:v for (k,v) in x2.items() if v in x4}
  x3.update(x5)
  return list(x3.keys())

In [48]:
diagnosis_icd9category_token['predicted_icd']=diagnosis_icd9category_token.predicted_icd_weigths.progress_apply(predictbasedonwts)

In [49]:
diagnosis_icd9category_token

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list,med_terms_condensed,features,predicted_icd_weigths,predicted_icd
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","[496, 553, 518, 453, 519, 276]","[hypercholesterolemia, cerebral, hypothyroidism, cough, accident, hypertension, artery, chest, vascular, coronary, fracture, injury, angina, pain, head]","[hypertension, fracture, artery, chest, pain, coronary, cough]","[f0, f10, f36, f17, f16, f24]","{'401': 0.02777777777777778, '427': 0.026041666666666668, '276': 0.0196078431372549, '414': 0.026041666666666668, '272': 0.026881720430107527, '250': 0.026041666666666668, '428': 0.025252525252525252, '518': 0.02314814814814815, '285': 0.02380952380952381, '584': 0.020833333333333332}","[401, 272, 427, 414, 250, 428, 285, 518, 584, 276]"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parking: main garage","[487, 416, 272, 530, 414, 553, 531, 5

In [50]:
result_predict={'category':[],'precision':[],'recall':[],'f1':[]}
for i in frequent_icd9category:
  actual_p=diagnosis_icd9category_token.loc[diagnosis_icd9category_token.icd9_category.apply(lambda x:i in x)].HADM_ID.count()
  tp=diagnosis_icd9category_token.loc[(diagnosis_icd9category_token.icd9_category.apply(lambda x:i in x))&(diagnosis_icd9category_token.predicted_icd.apply(lambda x:i in x))].HADM_ID.count()
  predict_p=diagnosis_icd9category_token.loc[diagnosis_icd9category_token.predicted_icd.apply(lambda x:i in x)].HADM_ID.count()
  precision=tp/predict_p
  recall=tp/actual_p
  f1=2*precision*recall/(precision+recall)
  result_predict['category'].append(i)
  result_predict['precision'].append(precision)
  result_predict['recall'].append(recall)
  result_predict['f1'].append(f1)

In [51]:
pd.DataFrame.from_dict(result_predict)

,category,precision,recall,f1
0,401,0.432877,0.854584,0.574665
1,427,0.346660,0.853080,0.492988
2,276,0.312711,0.837563,0.455396
3,414,0.292055,0.874852,0.437918
4,272,0.320960,0.879334,0.470270
5,250,0.296993,0.866474,0.442362
6,428,0.279617,0.870110,0.423227
7,518,0.256174,0.807617,0.388969
8,285,0.291350,0.892718,0.439322
9,584,0.239515,0.834991,0.372251


In [52]:
total_predicted=len([a for b in diagnosis_icd9category_token.predicted_icd.to_list() for a in b])
print(total_predicted)
actual=len([a for b in diagnosis_icd9category_token.icd9_category.apply(lambda x : [i for i in x if i in frequent_icd9category]).to_list() for a in b])
print(actual)
actual_predicted=0
for i in frequent_icd9category:
  actual_predicted += diagnosis_icd9category_token.loc[(diagnosis_icd9category_token.icd9_category.apply(lambda x:i in x))&(diagnosis_icd9category_token.predicted_icd.apply(lambda x:i in x))].HADM_ID.count()
print(actual_predicted)
recall=actual_predicted/actual
presicion=actual_predicted/total_predicted
print(recall)
print(presicion)
print(2*presicion*recall/(presicion + recall))

352853
126319
108297
0.8573294595429033
0.30691817839156815
0.4520172297212692


In [56]:
from sklearn.metrics import accuracy_score, hamming_loss
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score,precision_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer

In [58]:
mlb = MultiLabelBinarizer(frequent_icd9category)
print(accuracy_score(mlb.fit_transform(diagnosis_icd9category_token.icd9_category.values),mlb.fit_transform(diagnosis_icd9category_token.predicted_icd.values)))
print(hamming_loss(mlb.fit_transform(diagnosis_icd9category_token.icd9_category.values),mlb.fit_transform(diagnosis_icd9category_token.predicted_icd.values)))
print(precision_score(mlb.fit_transform(diagnosis_icd9category_token.icd9_category.values),mlb.fit_transform(diagnosis_icd9category_token.predicted_icd.values), average = "micro"))
print(recall_score(mlb.fit_transform(diagnosis_icd9category_token.icd9_category.values),mlb.fit_transform(diagnosis_icd9category_token.predicted_icd.values),average = "micro"))
print(f1_score(mlb.fit_transform(diagnosis_icd9category_token.icd9_category.values),mlb.fit_transform(diagnosis_icd9category_token.predicted_icd.values),average = "micro"))
print(classification_report(mlb.fit_transform(diagnosis_icd9category_token.icd9_category.values),mlb.fit_transform(diagnosis_icd9category_token.predicted_icd.values),target_names=mlb.classes_))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['003', '004', '005', '007', '008', '009', '010', '011', '012', '013', '014', '015', '018', '021', '023', '027', '030', '031', '032', '033', '034', '035', '036', '038', '039', '040', '041', '042', '045', '046', '047', '048', '049', '052', '053', '054', '057', '058', '062', '066', '070', '075', '077', '078', '079', '082', '084', '086', '088', '091', '093', '094', '096', '097', '099', '110', '111', '112', '114', '115', '116', '117', '118', '120', '121', '122', '123', '125', '127', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '161', '162', '163', '164', '170', '171', '172', '173', '174', '175', '176', '179', '180', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '19

0.019920413658350608
0.6256773178926299
0.30691817839156815
0.8573294595429033
0.4520172297212692
              precision    recall  f1-score   support

         401       0.43      0.85      0.57     17866
         427       0.35      0.85      0.49     14382
         276       0.31      0.84      0.46     13162
         414       0.29      0.87      0.44     11786
         272       0.32      0.88      0.47     12862
         250       0.30      0.87      0.44     12095
         428       0.28      0.87      0.42     11325
         518       0.26      0.81      0.39     11238
         285       0.29      0.89      0.44     11549
         584       0.24      0.83      0.37     10054

   micro avg       0.31      0.86      0.45    126319
   macro avg       0.31      0.86      0.45    126319
weighted avg       0.32      0.86      0.46    126319
 samples avg       0.26      0.76      0.37    126319



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
adm_icd = pd.read_csv("/content/drive/My Drive/AlgoIntern/Data/Data1.0/adm_icd.csv")

In [61]:
diagnosis_icd9category_token

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list,med_terms_condensed,features,predicted_icd_weigths,predicted_icd
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","[496, 553, 518, 453, 519, 276]","[hypercholesterolemia, cerebral, hypothyroidism, cough, accident, hypertension, artery, chest, vascular, coronary, fracture, injury, angina, pain, head]","[hypertension, fracture, artery, chest, pain, coronary, cough]","[f0, f10, f36, f17, f16, f24]","{'401': 0.02777777777777778, '427': 0.026041666666666668, '276': 0.0196078431372549, '414': 0.026041666666666668, '272': 0.026881720430107527, '250': 0.026041666666666668, '428': 0.025252525252525252, '518': 0.02314814814814815, '285': 0.02380952380952381, '584': 0.020833333333333332}","[401, 272, 427, 414, 250, 428, 285, 518, 584, 276]"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parking: main garage","[487, 416, 272, 530, 414, 553, 531, 5

In [62]:
diagnosis_icd10category=pd.merge(diagnosis_icd9category_token[['HADM_ID','discharge diagnosis:', 'medical_terms_list']],adm_icd[['HADM_ID','ICD10']],on="HADM_ID",how="left")

In [63]:
diagnosis_icd10category

,HADM_ID,discharge diagnosis:,medical_terms_list,ICD10
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","[hypercholesterolemia, cerebral, hypothyroidism, cough, accident, hypertension, artery, chest, vascular, coronary, fracture, injury, angina, pain, head]","J449,Misc,E872"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parking: main garage","[pulmonary, hypertension, of, obstructive, discomfort, gastrointestinal, constipation, oxygen, breath, inflammation, artery, chest, hydrochlorothiazide, steroid, anxiety, omeprazole, senna, trazodone, failure, shortness, disease, pantoprazole, aspirin, hypothyroidism, zolpidem, furosemide, coronary, prednisone, lethargy, steroids, ulcer, chronic, respiratory, morphine, pain]","I2510,E872,I9581,N179,E039,E784,I10,I952,Misc,E870,E785"
2,135453.0,discharge diagnosis: 1. cervical spondylosis with calcification of posterior longitudinal ligament. 2. fracture disl

In [64]:
diagnosis_icd10category['ICD10']=diagnosis_icd10category.ICD10.apply(lambda x:list(set(re.split(",",x))))

In [65]:
def icd10codes_into_category(x):
    return list(set([re.split("([A-Z]\d{2})",i)[1] for i in x]))

In [66]:
diagnosis_icd10category['ICD10']=diagnosis_icd10category.ICD10.apply(lambda x: [a for a in x if a!="Misc"])

In [67]:
diagnosis_icd10category['icd10category']=diagnosis_icd10category.ICD10.apply(icd10codes_into_category)

In [68]:
diagnosis_icd10category

,HADM_ID,discharge diagnosis:,medical_terms_list,ICD10,icd10category
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","[hypercholesterolemia, cerebral, hypothyroidism, cough, accident, hypertension, artery, chest, vascular, coronary, fracture, injury, angina, pain, head]","[J449, E872]","[E87, J44]"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parking: main garage","[pulmonary, hypertension, of, obstructive, discomfort, gastrointestinal, constipation, oxygen, breath, inflammation, artery, chest, hydrochlorothiazide, steroid, anxiety, omeprazole, senna, trazodone, failure, shortness, disease, pantoprazole, aspirin, hypothyroidism, zolpidem, furosemide, coronary, prednisone, lethargy, steroids, ulcer, chronic, respiratory, morphine, pain]","[E039, N179, I2510, I952, I9581, E785, E872, E784, I10, E870]","[N17, E87, I95, I25, I10, E78, E03]"
2,135453.0,discharge diagnosis: 1. cervical spondylosis wi

In [69]:
frequent_icd10category = ['I10', 'I25', 'E78', 'I50', 'I48', 'N17', 'E87', 'E11', 'J96', 'N39']

In [70]:
category_med_terms={}
common_diagnosis_terms=["monday","telephone","of","phone"]
for i in frequent_icd10category:
    diagnosis_x=diagnosis_icd10category.loc[diagnosis_icd10category.icd10category.apply(lambda x: i in x)].copy()
    y=diagnosis_x.medical_terms_list.to_list()    
    z=[list(set(a)) for a in y]
    z1=[a for b in z for a in b]
    dict_med=dict(Counter(z1))
    dict_med=dict(sorted(dict_med.items(),key= lambda x: x[1],reverse=True))
    dict_med_list=list(dict_med)
    dict_med_list1=[a for a in dict_med_list if a not in common_diagnosis_terms]
    category_med_terms[i]=dict_med_list1[:50]

diagnosis_icdcategory_medterms={'icd10category':[i for i in category_med_terms],
                                'frequent_medterms':[category_med_terms[i] for i in category_med_terms]}
diagnosis_icdcategory_medterms_df=pd.DataFrame.from_dict(diagnosis_icdcategory_medterms)

diagnosis_icdcategory_medterms_df

,icd10category,frequent_medterms
0,I10,"[pain, fever, chest, shortness, infection, breath, redness, aspirin, hypertension, atrial, fibrillation, chills, disease, coumadin, bleeding, failure, pneumonia, vomiting, fevers, constipation, swelling, nausea, abdominal, heart, diabetes, erythema, cough, weakness, anemia, metoprolol, headache, confusion, mouth, numbness, weight, diarrhea, gain, lisinopril, loss, cancer, blood, infarction, seizures, pulmonary, myocardial, stroke, bleed, motrin, artery, tylenol]"
1,I25,"[pain, chest, fever, shortness, infection, breath, disease, redness, atrial, failure, fibrillation, aspirin, heart, coumadin, hypertension, myocardial, infarction, erythema, bleeding, weight, chills, gain, diabetes, metoprolol, artery, renal, coronary, pneumonia, fevers, lisinopril, anemia, vomiting, sodium, chronic, nausea, mellitus, abdominal, constipation, pulmonary, swelling, edema, cough, mouth, congestive, cholesterol, bleed, cancer, blood, loss, diarrhea]"
2,E78,"[pain, fever, chest, infection, shortness, breath, redness, aspirin, atrial, disease, fibrillation, failure, coumadin, hypertension, bleeding, chills, heart, erythema, pneumonia, diabetes, constipation, vomiting, fevers, metoprolol, swelling, weight, renal, gain, nausea, anemia, abdominal, infarction, myocardial, lisinopril, cough, mouth, weakness, cancer, artery, confusion, mellitus, headache, pulmonary, loss, chronic, hyperlipidemia, diarrhea, blood, numbness, tylenol]"
3,I50,"[pain, failure, shortness, chest, heart, breath, atrial, fibrillation, disease, infection, fever, pneumonia, coumadin, chills, renal, hypertension, fevers, sodium, aspirin, metoprolol, bleeding, chronic, anemia, congestive, diabetes, redness, pulmonary, vomiting, nausea, lisinopril, abdominal, oxygen, myocardial, infarction, respiratory, artery, systolic, coronary, mouth, urinary, cough, blood, mellitus, swelling, tract, bleed, weight, acute, diarrhea, gain]"
4,I48,"[atrial, pain, fibrillation, coumadin, shortness, chest, fever, failure, infection, breath, disease, heart, redness, pneumonia, chills, aspirin, hypertension, bleeding, metoprolol, renal, fevers, anemia, erythema, vomiting, abdominal, sodium, weight, gain, nausea, chronic, diabetes, pulmonary, warfarin, constipation, bleed, stroke, cough, swelling, cancer, mouth, diarrhea, artery, urinary, congestive, blood, tract, respiratory, oxygen, lisinopril, amiodarone]"
5,N17,"[pain, failure, shortness, chest, infection, breath, renal, disease, pneumonia, fever, atrial, chills, fibrillation, heart, fevers, chronic, hypertension, vomiting, bleeding, acute, abdominal, coumadin, anemia, nausea, diabetes, aspirin, metoprolol, kidney, diarrhea, sodium, respiratory, mouth, urinary, lisinopril, tract, blood, pulmonary, cough, cancer, redness, sepsis, swelling, oxygen, constipation, bleed, loss, confusion, congestive, mellitus, dizziness]"
6,E87,"[pain, infection, shortness, failure, chest, breath, pneumonia, fever, disease, chills, atrial, fibrillation, fevers, renal, bleeding, vomiting, hypertension, abdominal, heart, nausea, aspirin, anemia, coumadin, chronic, diabetes, diarrhea, metoprolol, respiratory, mouth, sodium, constipation, redness, confusion, urinary, swelling, acute, cough, tract, alcohol, blood, pulmonary, cancer, lisinopril, oxygen, kidney, loss, bleed, sepsis, weakness, seizures]"
7,E11,"[pain, shortness, chest, fever, breath, infection, failure, diabetes, disease, chills, atrial, fibrillation, redness, heart, aspirin, hypertension, bleeding, coumadin, fevers, pneumonia, mellitus, vomiting, renal, nausea, abdominal, metoprolol, anemia, constipation, swelling, chronic, sodium, cough, erythema, mouth, lisinopril, diarrhea, blood, confusion, artery, weight, pulmonary, infarction, gain, bleed, weakness, myocardial, respiratory, oxygen, urinary, loss]"
8,J96,"[pain, pneumonia, failure, shortness, breath, chest, respiratory, infection, disease, fever, chills, atrial, fibrillation, fevers, heart, hypertension, renal, vomiting, 

In [71]:
bagofwords=list(set([a for b in diagnosis_icdcategory_medterms_df.frequent_medterms.to_list() for a in b]))

In [72]:
simi_matrix=np.zeros((len(bagofwords),len(bagofwords)))

In [73]:
simi_matrix.shape

(76, 76)

In [74]:
for i in tqdm(range(len(bagofwords))):
  for j in range(len(bagofwords)):
    simi_matrix[i][j]=model.similarity(bagofwords[i],bagofwords[j])

100%|██████████| 76/76 [00:00<00:00, 585.72it/s]


In [75]:
features={}
for i in range(len(bagofwords)-1):
  similar=[]
  for j in range(i+1,len(bagofwords)):
    if simi_matrix[i][j]>=0.7:
      similar.append(bagofwords[j])
  features[bagofwords[i]]=similar
features[bagofwords[-1]]=[]    


In [76]:
len(features)

76

In [77]:
features_list=[features[k]+[k] for k in list(features)]

In [78]:
condensed_feature = list(merge_common(features_list)) 

In [79]:
condensed_feature_dict={}
count=0
for i in condensed_feature:
  condensed_feature_dict['f'+str(count)]=i
  count+=1

In [80]:
all_vocab=[a for b in condensed_feature for a in b]

In [81]:
diagnosis_icd10category['med_terms_condensed']=diagnosis_icd10category.medical_terms_list.progress_apply(lambda x:list(set(x).intersection(set(all_vocab))))
diagnosis_icd10category['features']=diagnosis_icd10category.med_terms_condensed.progress_apply(lambda x:return_feature(x))

In [82]:
diagnosis_icdcategory_medterms_df['feature']=diagnosis_icdcategory_medterms_df.frequent_medterms.progress_apply(lambda x:return_feature(x))

In [83]:
def predict_icd(x):
  category_weights={}
  if len(x):
    for item1 in frequent_icd10category:
      category_features=[a for b in diagnosis_icdcategory_medterms_df.loc[diagnosis_icdcategory_medterms_df.icd10category==item1].feature.to_list() for a in b]
      count=len(list(set(x).intersection(set(category_features))))
      category_weights[item1]=count/len(x)/len(category_features)
  else:
    for item1 in frequent_icd10category:
      category_weights[item1]=0.0

      
  return category_weights
  

In [84]:
diagnosis_icd10category['predicted_icd_weigths']=diagnosis_icd10category.features.progress_apply(lambda x:predict_icd(x))
diagnosis_icd10category['predicted_icd']=diagnosis_icd10category.predicted_icd_weigths.progress_apply(predictbasedonwts)

In [85]:
result_predict={'category':[],'precision':[],'recall':[],'f1':[]}
for i in frequent_icd10category:
  actual_p=diagnosis_icd10category.loc[diagnosis_icd10category.icd10category.apply(lambda x:i in x)].HADM_ID.count()
  tp=diagnosis_icd10category.loc[(diagnosis_icd10category.icd10category.apply(lambda x:i in x))&(diagnosis_icd10category.predicted_icd.apply(lambda x:i in x))].HADM_ID.count()
  predict_p=diagnosis_icd10category.loc[diagnosis_icd10category.predicted_icd.apply(lambda x:i in x)].HADM_ID.count()
  precision=tp/predict_p
  recall=tp/actual_p
  f1=2*precision*recall/(precision+recall)
  result_predict['category'].append(i)
  result_predict['precision'].append(precision)
  result_predict['recall'].append(recall)
  result_predict['f1'].append(f1)

In [86]:
pd.DataFrame.from_dict(result_predict)

,category,precision,recall,f1
0,I10,0.418209,0.853474,0.561351
1,I25,0.299332,0.875290,0.446104
2,E78,0.318987,0.879665,0.468195
3,I50,0.270710,0.870620,0.413002
4,I48,0.274746,0.867231,0.417291
5,N17,0.238269,0.835049,0.370750
6,E87,0.227958,0.830825,0.357756
7,E11,0.223692,0.863975,0.355375
8,J96,0.139011,0.783100,0.236110
9,N39,0.139946,0.877344,0.241388


In [87]:
total_predicted=len([a for b in diagnosis_icd10category.predicted_icd.to_list() for a in b])
print(total_predicted)
actual=len([a for b in diagnosis_icd10category.icd10category.apply(lambda x : [i for i in x if i in frequent_icd10category]).to_list() for a in b])
print(actual)
actual_predicted=0
for i in frequent_icd10category:
  actual_predicted += diagnosis_icd10category.loc[(diagnosis_icd10category.icd10category.apply(lambda x:i in x))&(diagnosis_icd10category.predicted_icd.apply(lambda x:i in x))].HADM_ID.count()
print(actual_predicted)
recall=actual_predicted/actual
presicion=actual_predicted/total_predicted
print(recall)
print(presicion)
print(2*presicion*recall/(presicion + recall))

353367
105151
90070
0.8565776835217924
0.2548908075739953
0.3928744345914446


In [88]:
mlb = MultiLabelBinarizer(frequent_icd10category)
print(accuracy_score(mlb.fit_transform(diagnosis_icd10category.icd10category.values),mlb.fit_transform(diagnosis_icd10category.predicted_icd.values)))
print(hamming_loss(mlb.fit_transform(diagnosis_icd10category.icd10category.values),mlb.fit_transform(diagnosis_icd10category.predicted_icd.values)))
print(precision_score(mlb.fit_transform(diagnosis_icd10category.icd10category.values),mlb.fit_transform(diagnosis_icd10category.predicted_icd.values), average = "micro"))
print(recall_score(mlb.fit_transform(diagnosis_icd10category.icd10category.values),mlb.fit_transform(diagnosis_icd10category.predicted_icd.values),average = "micro"))
print(f1_score(mlb.fit_transform(diagnosis_icd10category.icd10category.values),mlb.fit_transform(diagnosis_icd10category.predicted_icd.values),average = "micro"))
print(classification_report(mlb.fit_transform(diagnosis_icd10category.icd10category.values),mlb.fit_transform(diagnosis_icd10category.predicted_icd.values),target_names=mlb.classes_))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['A04', 'A41', 'B18', 'D62', 'D63', 'D64', 'D69', 'E03', 'E08', 'E09', 'E10', 'E13', 'E46', 'E66', 'E86', 'F05', 'F17', 'F32', 'F41', 'G47', 'I12', 'I20', 'I21', 'I27', 'I34', 'I35', 'I42', 'I46', 'I47', 'I49', 'I61', 'I73', 'I95', 'I97', 'J18', 'J44', 'J45', 'J69', 'J91', 'J98', 'K21', 'K74', 'K92', 'L89', 'M10', 'M81', 'N18', 'N40', 'P00', 'P22', 'P59', 'R00', 'R09', 'R56', 'R65', 'R78', 'Y83', 'Y84', 'Z23', 'Z38', 'Z41', 'Z66', 'Z79', 'Z85', 'Z86', 'Z87', 'Z95', 'Z98'] will be ignored
  .format(sorted(unknown, key=str)))


0.02556770796101699
0.6633259465770724
0.2548908075739953
0.8565776835217924
0.3928744345914446
              precision    recall  f1-score   support

         I10       0.42      0.85      0.56     17287
         I25       0.30      0.88      0.45     12076
         E78       0.32      0.88      0.47     12781
         I50       0.27      0.87      0.41     10960
         I48       0.27      0.87      0.42     11215
         N17       0.24      0.84      0.37     10003
         E87       0.23      0.83      0.36      9700
         E11       0.22      0.86      0.36      9138
         J96       0.14      0.78      0.24      6284
         N39       0.14      0.88      0.24      5707

   micro avg       0.25      0.86      0.39    105151
   macro avg       0.26      0.85      0.39    105151
weighted avg       0.28      0.86      0.41    105151
 samples avg       0.22      0.72      0.32    105151



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [91]:
diagnosis_terms_codes=pd.merge(diagnosis_icd9category_token[['HADM_ID','discharge diagnosis:', 'medical_terms_list']],adm_icd,on="HADM_ID",how="left")

In [92]:
diagnosis_terms_codes['ICD9_CODE']=diagnosis_terms_codes.ICD9_CODE.apply(lambda x:list(set(re.split(",",x))))
diagnosis_terms_codes['ICD10']=diagnosis_terms_codes.ICD10.apply(lambda x:list(set(re.split(",",x))))

In [93]:
diagnosis_terms_codes['ICD10']=diagnosis_terms_codes.ICD10.apply(lambda x:[a for a in x if a!="Misc"])

In [94]:
frequent_icd9codes = ['4019', '4280', '42731', '41401', '5849', '25000', '2724', '51881', '5990', '53081']

In [95]:
code_med_terms={}
common_diagnosis_terms=["monday","telephone","of","phone"]
for i in frequent_icd9codes:
    diagnosis_x=diagnosis_terms_codes.loc[diagnosis_terms_codes.ICD9_CODE.apply(lambda x: i in x)].copy()
    y=diagnosis_x.medical_terms_list.to_list()    
    z=[list(set(a)) for a in y]
    z1=[a for b in z for a in b]
    dict_med=dict(Counter(z1))
    dict_med=dict(sorted(dict_med.items(),key= lambda x: x[1],reverse=True))
    dict_med_list=list(dict_med)
    dict_med_list1=[a for a in dict_med_list if a not in common_diagnosis_terms]
    code_med_terms[i]=dict_med_list1[:50]

diagnosis_icdcategory_medterms={'icd9code':[i for i in code_med_terms],
                                'frequent_medterms':[code_med_terms[i] for i in code_med_terms]}
diagnosis_icdcategory_medterms_df=pd.DataFrame.from_dict(diagnosis_icdcategory_medterms)

display(diagnosis_icdcategory_medterms_df)

bagofwords=list(set([a for b in diagnosis_icdcategory_medterms_df.frequent_medterms.to_list() for a in b]))
simi_matrix=np.zeros((len(bagofwords),len(bagofwords)))

for i in tqdm(range(len(bagofwords))):
  for j in range(len(bagofwords)):
    simi_matrix[i][j]=model.similarity(bagofwords[i],bagofwords[j])

features={}
for i in range(len(bagofwords)-1):
  similar=[]
  for j in range(i+1,len(bagofwords)):
    if simi_matrix[i][j]>=0.7:
      similar.append(bagofwords[j])
  features[bagofwords[i]]=similar
features[bagofwords[-1]]=[]    

features_list=[features[k]+[k] for k in list(features)]

condensed_feature = list(merge_common(features_list)) 

condensed_feature_dict={}
count=0
for i in condensed_feature:
  condensed_feature_dict['f'+str(count)]=i
  count+=1

all_vocab=[a for b in condensed_feature for a in b]

diagnosis_terms_codes['med_terms_condensed']=diagnosis_terms_codes.medical_terms_list.progress_apply(lambda x:list(set(x).intersection(set(all_vocab))))
diagnosis_terms_codes['features']=diagnosis_terms_codes.med_terms_condensed.progress_apply(lambda x:return_feature(x))

diagnosis_icdcategory_medterms_df['feature']=diagnosis_icdcategory_medterms_df.frequent_medterms.progress_apply(lambda x:return_feature(x))


def predict_icd(x):
  category_weights={}
  if len(x):
    for item1 in frequent_icd9codes:
      category_features=[a for b in diagnosis_icdcategory_medterms_df.loc[diagnosis_icdcategory_medterms_df.icd9code==item1].feature.to_list() for a in b]
      count=len(list(set(x).intersection(set(category_features))))
      category_weights[item1]=count/len(x)/len(category_features)
  else:
    for item1 in frequent_icd9codes:
      category_weights[item1]=0.0

      
  return category_weights



diagnosis_terms_codes['predicted_icd_weigths']=diagnosis_terms_codes.features.progress_apply(lambda x:predict_icd(x))
diagnosis_terms_codes['predicted_icd']=diagnosis_terms_codes.predicted_icd_weigths.progress_apply(predictbasedonwts)


result_predict={'category':[],'precision':[],'recall':[],'f1':[]}
for i in frequent_icd9codes:
  actual_p=diagnosis_terms_codes.loc[diagnosis_terms_codes.ICD9_CODE.apply(lambda x:i in x)].HADM_ID.count()
  tp=diagnosis_terms_codes.loc[(diagnosis_terms_codes.ICD9_CODE.apply(lambda x:i in x))&(diagnosis_terms_codes.predicted_icd.apply(lambda x:i in x))].HADM_ID.count()
  predict_p=diagnosis_terms_codes.loc[diagnosis_terms_codes.predicted_icd.apply(lambda x:i in x)].HADM_ID.count()
  precision=tp/predict_p
  recall=tp/actual_p
  f1=2*precision*recall/(precision+recall)
  result_predict['category'].append(i)
  result_predict['precision'].append(precision)
  result_predict['recall'].append(recall)
  result_predict['f1'].append(f1)



display(pd.DataFrame.from_dict(result_predict))


total_predicted=len([a for b in diagnosis_terms_codes.predicted_icd.to_list() for a in b])
print(total_predicted)
actual=len([a for b in diagnosis_terms_codes.ICD9_CODE.apply(lambda x : [i for i in x if i in frequent_icd9codes]).to_list() for a in b])
print(actual)
actual_predicted=0
for i in frequent_icd9codes:
  actual_predicted += diagnosis_terms_codes.loc[(diagnosis_terms_codes.ICD9_CODE.apply(lambda x:i in x))&(diagnosis_terms_codes.predicted_icd.apply(lambda x:i in x))].HADM_ID.count()
print(actual_predicted)
recall=actual_predicted/actual
presicion=actual_predicted/total_predicted
print(recall)
print(presicion)
print(2*presicion*recall/(presicion + recall))


,icd9code,frequent_medterms
0,4019,"[pain, fever, chest, shortness, infection, breath, redness, aspirin, hypertension, atrial, fibrillation, chills, disease, coumadin, bleeding, failure, pneumonia, vomiting, fevers, constipation, swelling, nausea, abdominal, heart, diabetes, erythema, cough, weakness, anemia, metoprolol, headache, confusion, mouth, numbness, weight, diarrhea, gain, lisinopril, loss, cancer, blood, infarction, seizures, pulmonary, myocardial, stroke, bleed, motrin, artery, tylenol]"
1,4280,"[pain, failure, shortness, chest, heart, breath, atrial, fibrillation, disease, infection, fever, pneumonia, coumadin, chills, hypertension, renal, fevers, sodium, aspirin, metoprolol, bleeding, chronic, anemia, congestive, diabetes, redness, pulmonary, vomiting, nausea, lisinopril, abdominal, oxygen, myocardial, infarction, respiratory, artery, systolic, coronary, mouth, cough, urinary, blood, mellitus, swelling, tract, bleed, weight, acute, diarrhea, gain]"
2,42731,"[atrial, fibrillation, pain, coumadin, shortness, fever, chest, infection, failure, breath, disease, heart, redness, pneumonia, chills, aspirin, hypertension, bleeding, metoprolol, renal, fevers, anemia, erythema, vomiting, abdominal, sodium, weight, gain, nausea, chronic, diabetes, pulmonary, warfarin, stroke, constipation, bleed, cough, swelling, cancer, diarrhea, mouth, urinary, artery, congestive, blood, tract, oxygen, lisinopril, respiratory, coronary]"
3,41401,"[pain, fever, chest, shortness, redness, infection, disease, breath, atrial, fibrillation, failure, aspirin, heart, myocardial, infarction, hypertension, coumadin, erythema, weight, gain, diabetes, artery, metoprolol, coronary, bleeding, chills, renal, pneumonia, fevers, lisinopril, mellitus, anemia, sodium, chronic, nausea, vomiting, abdominal, pulmonary, edema, cholesterol, cancer, constipation, swelling, congestive, bleed, hyperlipidemia, cough, atorvastatin, mouth, blood]"
4,5849,"[pain, failure, shortness, chest, breath, infection, disease, renal, pneumonia, fever, chills, atrial, heart, fibrillation, fevers, chronic, hypertension, coumadin, bleeding, vomiting, anemia, abdominal, nausea, diabetes, acute, aspirin, metoprolol, sodium, diarrhea, kidney, urinary, lisinopril, mouth, respiratory, tract, blood, pulmonary, cough, cancer, oxygen, bleed, redness, swelling, sepsis, constipation, congestive, loss, confusion, dizziness, mellitus]"
5,25000,"[pain, shortness, chest, fever, breath, infection, failure, diabetes, disease, atrial, fibrillation, chills, redness, aspirin, heart, hypertension, bleeding, coumadin, pneumonia, fevers, mellitus, vomiting, renal, abdominal, metoprolol, nausea, constipation, swelling, anemia, cough, sodium, chronic, erythema, mouth, lisinopril, diarrhea, pulmonary, weight, confusion, artery, gain, bleed, weakness, blood, infarction, respiratory, oxygen, myocardial, cancer, loss]"
6,2724,"[pain, infection, chest, fever, shortness, redness, breath, aspirin, atrial, disease, fibrillation, failure, coumadin, hypertension, bleeding, heart, erythema, pneumonia, chills, metoprolol, diabetes, constipation, vomiting, fevers, swelling, weight, renal, lisinopril, gain, mouth, nausea, infarction, hyperlipidemia, myocardial, anemia, cough, abdominal, tylenol, pulmonary, cancer, mellitus, blood, chronic, artery, weakness, stroke, loss, edema, diarrhea, confusion]"
7,51881,"[pain, pneumonia, failure, shortness, breath, chest, respiratory, infection, disease, fever, chills, atrial, fibrillation, fevers, heart, hypertension, renal, vomiting, oxygen, pulmonary, nausea, anemia, cough, abdominal, aspirin, coumadin, bleeding, chronic, metoprolol, mouth, diarrhea, diabetes, sepsis, cancer, constipation, confusion, acute, sodium, swelling, prednisone, arrest, urinary, weakness, blood, lisinopril, lung, redness, tract, obstructive, seizures]"
8,5990,"[pain, infection, urinary, tract, shortness, chest, breath, fever, failure, disease, atrial, fibrillation, chills, pneumonia, fevers, bleeding, as

100%|██████████| 75/75 [00:00<00:00, 664.34it/s]


,category,precision,recall,f1
0,4019,0.418235,0.853358,0.561350
1,4280,0.269253,0.869992,0.411233
2,42731,0.266684,0.866520,0.407847
3,41401,0.249306,0.878319,0.388374
4,5849,0.195995,0.847730,0.318380
5,25000,0.186916,0.859126,0.307033
6,2724,0.214788,0.902012,0.346958
7,51881,0.139003,0.782941,0.236090
8,5990,0.139970,0.877344,0.241423
9,53081,0.139486,0.882121,0.240883


353427
90890
78399
0.8625701397293432
0.2218251576704666
0.35289669312675415


In [96]:
mlb = MultiLabelBinarizer(frequent_icd9codes)
print(accuracy_score(mlb.fit_transform(diagnosis_terms_codes.ICD9_CODE.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values)))
print(hamming_loss(mlb.fit_transform(diagnosis_terms_codes.ICD9_CODE.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values)))
print(precision_score(mlb.fit_transform(diagnosis_terms_codes.ICD9_CODE.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values), average = "micro"))
print(recall_score(mlb.fit_transform(diagnosis_terms_codes.ICD9_CODE.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values),average = "micro"))
print(f1_score(mlb.fit_transform(diagnosis_terms_codes.ICD9_CODE.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values),average = "micro"))
print(classification_report(mlb.fit_transform(diagnosis_terms_codes.ICD9_CODE.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values),target_names=mlb.classes_))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['0030', '0031', '0038', '0039', '0041', '0048', '0049', '0051', '00581', '0059', '0071', '0074', '00804', '00841', '00843', '00845', '00847', '0085', '00861', '00862', '00863', '00869', '0088', '0090', '0091', '0092', '0093', '01085', '01123', '01164', '01186', '01190', '01193', '01194', '01205', '01215', '01300', '01304', '01325', '01330', '01354', '01402', '01405', '01485', '01504', '01505', '01803', '01805', '01880', '01890', '01894', '01895', '01896', '0218', '0239', '0270', '0272', '0279', '0309', '0310', '0311', '0312', '0318', '0319', '03285', '03289', '0329', '0338', '0340', '035', '0360', '0362', '03641', '0380', '03810', '03811', '03812', '03819', '0382', '0383', '03840', '03841', '03842', '03843', '03844', '03849', '0388', '0389', '0391', '0398', '0400', '04082', '04100', '04101', '04102', '04103', '04104', '04105', '04109', '04110', '04111', '04112', '04119', '0412', 

0.030857578573641194
0.6851073462482427
0.2218251576704666
0.8625701397293432
0.35289669312675415
              precision    recall  f1-score   support

        4019       0.42      0.85      0.56     17287
        4280       0.27      0.87      0.41     10907
       42731       0.27      0.87      0.41     10893
       41401       0.25      0.88      0.39     10018
        5849       0.20      0.85      0.32      8104
       25000       0.19      0.86      0.31      7709
        2724       0.21      0.90      0.35      8399
       51881       0.14      0.78      0.24      6284
        5990       0.14      0.88      0.24      5707
       53081       0.14      0.88      0.24      5582

   micro avg       0.22      0.86      0.35     90890
   macro avg       0.22      0.86      0.35     90890
weighted avg       0.25      0.86      0.38     90890
 samples avg       0.19      0.70      0.28     90890



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [140]:
frequent_icd10codes = ['I10', 'I2510', 'I509', 'I4891', 'N179', 'E119', 'E784', 'E785', 'J9690', 'J9600']

In [141]:
code_med_terms={}
common_diagnosis_terms=["monday","telephone","of","phone"]
for i in frequent_icd10codes:
    diagnosis_x=diagnosis_terms_codes.loc[diagnosis_terms_codes.ICD10.apply(lambda x: i in x)].copy()
    y=diagnosis_x.medical_terms_list.to_list()    
    z=[list(set(a)) for a in y]
    z1=[a for b in z for a in b]
    dict_med=dict(Counter(z1))
    dict_med=dict(sorted(dict_med.items(),key= lambda x: x[1],reverse=True))
    dict_med_list=list(dict_med)
    dict_med_list1=[a for a in dict_med_list if a not in common_diagnosis_terms]
    code_med_terms[i]=dict_med_list1[:50]

diagnosis_icdcategory_medterms={'icd10code':[i for i in code_med_terms],
                                'frequent_medterms':[code_med_terms[i] for i in code_med_terms]}
diagnosis_icdcategory_medterms_df=pd.DataFrame.from_dict(diagnosis_icdcategory_medterms)

display(diagnosis_icdcategory_medterms_df)

bagofwords=list(set([a for b in diagnosis_icdcategory_medterms_df.frequent_medterms.to_list() for a in b]))
simi_matrix=np.zeros((len(bagofwords),len(bagofwords)))

for i in tqdm(range(len(bagofwords))):
  for j in range(len(bagofwords)):
    simi_matrix[i][j]=model.similarity(bagofwords[i],bagofwords[j])

features={}
for i in range(len(bagofwords)-1):
  similar=[]
  for j in range(i+1,len(bagofwords)):
    if simi_matrix[i][j]>=0.7:
      similar.append(bagofwords[j])
  features[bagofwords[i]]=similar
features[bagofwords[-1]]=[]    

features_list=[features[k]+[k] for k in list(features)]

condensed_feature = list(merge_common(features_list)) 

condensed_feature_dict={}
count=0
for i in condensed_feature:
  condensed_feature_dict['f'+str(count)]=i
  count+=1

all_vocab=[a for b in condensed_feature for a in b]

diagnosis_terms_codes['med_terms_condensed']=diagnosis_terms_codes.medical_terms_list.progress_apply(lambda x:list(set(x).intersection(set(all_vocab))))
diagnosis_terms_codes['features']=diagnosis_terms_codes.med_terms_condensed.progress_apply(lambda x:return_feature(x))

diagnosis_icdcategory_medterms_df['feature']=diagnosis_icdcategory_medterms_df.frequent_medterms.progress_apply(lambda x:return_feature(x))


def predict_icd(x):
  category_weights={}
  if len(x):
    for item1 in frequent_icd10codes:
      category_features=[a for b in diagnosis_icdcategory_medterms_df.loc[diagnosis_icdcategory_medterms_df.icd10code==item1].feature.to_list() for a in b]
      count=len(list(set(x).intersection(set(category_features))))
      category_weights[item1]=count/len(x)/len(category_features)
  else:
    for item1 in frequent_icd10codes:
      category_weights[item1]=0.0

      
  return category_weights



diagnosis_terms_codes['predicted_icd_weigths']=diagnosis_terms_codes.features.progress_apply(lambda x:predict_icd(x))
diagnosis_terms_codes['predicted_icd']=diagnosis_terms_codes.predicted_icd_weigths.progress_apply(predictbasedonwts)


result_predict={'category':[],'precision':[],'recall':[],'f1':[]}
for i in frequent_icd10codes:
  actual_p=diagnosis_terms_codes.loc[diagnosis_terms_codes.ICD10.apply(lambda x:i in x)].HADM_ID.count()
  tp=diagnosis_terms_codes.loc[(diagnosis_terms_codes.ICD10.apply(lambda x:i in x))&(diagnosis_terms_codes.predicted_icd.apply(lambda x:i in x))].HADM_ID.count()
  predict_p=diagnosis_terms_codes.loc[diagnosis_terms_codes.predicted_icd.apply(lambda x:i in x)].HADM_ID.count()
  precision=tp/predict_p
  recall=tp/actual_p
  f1=2*precision*recall/(precision+recall)
  result_predict['category'].append(i)
  result_predict['precision'].append(precision)
  result_predict['recall'].append(recall)
  result_predict['f1'].append(f1)



display(pd.DataFrame.from_dict(result_predict))


total_predicted=len([a for b in diagnosis_terms_codes.predicted_icd.to_list() for a in b])
print(total_predicted)
actual=len([a for b in diagnosis_terms_codes.ICD10.apply(lambda x : [i for i in x if i in frequent_icd10codes]).to_list() for a in b])
print(actual)
actual_predicted=0
for i in frequent_icd10codes:
  actual_predicted += diagnosis_terms_codes.loc[(diagnosis_terms_codes.ICD10.apply(lambda x:i in x))&(diagnosis_terms_codes.predicted_icd.apply(lambda x:i in x))].HADM_ID.count()
print(actual_predicted)
recall=actual_predicted/actual
presicion=actual_predicted/total_predicted
print(recall)
print(presicion)
print(2*presicion*recall/(presicion + recall))


,icd10code,frequent_medterms
0,I10,"[pain, fever, chest, shortness, infection, breath, redness, aspirin, hypertension, atrial, fibrillation, chills, disease, coumadin, bleeding, failure, pneumonia, vomiting, fevers, constipation, swelling, nausea, abdominal, heart, diabetes, erythema, cough, weakness, anemia, metoprolol, headache, confusion, mouth, numbness, weight, diarrhea, gain, lisinopril, loss, cancer, blood, infarction, seizures, pulmonary, myocardial, stroke, bleed, motrin, artery, tylenol]"
1,I2510,"[pain, chest, fever, shortness, infection, disease, breath, redness, atrial, fibrillation, failure, aspirin, heart, coumadin, hypertension, myocardial, infarction, erythema, weight, gain, bleeding, diabetes, artery, chills, metoprolol, coronary, renal, pneumonia, fevers, lisinopril, anemia, sodium, vomiting, chronic, mellitus, nausea, abdominal, pulmonary, constipation, edema, swelling, congestive, bleed, cholesterol, cancer, cough, mouth, blood, loss, hyperlipidemia]"
2,I509,"[pain, failure, shortness, chest, heart, breath, atrial, fibrillation, disease, infection, fever, pneumonia, coumadin, chills, hypertension, renal, fevers, sodium, aspirin, metoprolol, bleeding, chronic, anemia, congestive, diabetes, redness, pulmonary, vomiting, nausea, lisinopril, abdominal, oxygen, myocardial, infarction, respiratory, artery, systolic, coronary, mouth, cough, urinary, blood, mellitus, swelling, tract, bleed, weight, acute, diarrhea, gain]"
3,I4891,"[atrial, fibrillation, pain, coumadin, shortness, fever, chest, infection, failure, breath, disease, heart, redness, pneumonia, chills, aspirin, hypertension, bleeding, metoprolol, renal, fevers, anemia, erythema, vomiting, abdominal, sodium, weight, gain, nausea, chronic, diabetes, pulmonary, warfarin, stroke, constipation, bleed, cough, swelling, cancer, diarrhea, mouth, urinary, artery, congestive, blood, tract, oxygen, lisinopril, respiratory, coronary]"
4,N179,"[pain, failure, shortness, chest, breath, infection, disease, renal, pneumonia, fever, chills, atrial, heart, fibrillation, fevers, chronic, hypertension, coumadin, bleeding, vomiting, anemia, abdominal, nausea, diabetes, acute, aspirin, metoprolol, sodium, diarrhea, kidney, urinary, lisinopril, mouth, respiratory, tract, blood, pulmonary, cough, cancer, oxygen, bleed, redness, swelling, sepsis, constipation, congestive, loss, confusion, dizziness, mellitus]"
5,E119,"[pain, shortness, chest, fever, breath, infection, failure, diabetes, disease, atrial, fibrillation, chills, redness, aspirin, heart, hypertension, bleeding, coumadin, pneumonia, fevers, mellitus, vomiting, renal, abdominal, metoprolol, nausea, constipation, swelling, anemia, cough, sodium, chronic, erythema, mouth, lisinopril, diarrhea, pulmonary, weight, confusion, artery, gain, bleed, weakness, blood, infarction, respiratory, oxygen, myocardial, cancer, loss]"
6,E784,"[pain, infection, chest, fever, shortness, redness, breath, aspirin, atrial, disease, fibrillation, failure, coumadin, hypertension, bleeding, heart, erythema, pneumonia, chills, metoprolol, diabetes, constipation, vomiting, fevers, swelling, weight, renal, lisinopril, gain, mouth, nausea, infarction, hyperlipidemia, myocardial, anemia, cough, abdominal, tylenol, pulmonary, cancer, mellitus, blood, chronic, artery, weakness, stroke, loss, edema, diarrhea, confusion]"
7,E785,"[pain, infection, chest, fever, shortness, redness, breath, aspirin, atrial, disease, fibrillation, failure, coumadin, hypertension, bleeding, heart, erythema, pneumonia, chills, metoprolol, diabetes, constipation, vomiting, fevers, swelling, weight, renal, lisinopril, gain, mouth, nausea, infarction, hyperlipidemia, myocardial, anemia, cough, abdominal, tylenol, pulmonary, cancer, mellitus, blood, chronic, artery, weakness, stroke, loss, edema, diarrhea, confusion]"
8,J9690,"[pain, pneumonia, failure, shortness, breath, chest, respiratory, infection, disease, fever, chills, atrial, fibrillation, fevers, heart, hypertension, 

100%|██████████| 73/73 [00:00<00:00, 692.73it/s]


,category,precision,recall,f1
0,I10,0.418247,0.853358,0.561361
1,I2510,0.281575,0.876168,0.426186
2,I509,0.269232,0.869900,0.411199
3,I4891,0.266691,0.866520,0.407856
4,N179,0.195977,0.847606,0.318348
5,E119,0.186921,0.859126,0.307040
6,E784,0.214766,0.901893,0.346920
7,E785,0.214766,0.901893,0.346920
8,J9690,0.139011,0.782941,0.236101
9,J9600,0.139011,0.782941,0.236101


353020
95612
82102
0.8586997448019077
0.23257039261231657
0.36601044954439277


In [142]:
mlb = MultiLabelBinarizer(frequent_icd10codes)
print(accuracy_score(mlb.fit_transform(diagnosis_terms_codes.ICD10.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values)))
print(hamming_loss(mlb.fit_transform(diagnosis_terms_codes.ICD10.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values)))
print(precision_score(mlb.fit_transform(diagnosis_terms_codes.ICD10.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values), average = "micro"))
print(recall_score(mlb.fit_transform(diagnosis_terms_codes.ICD10.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values),average = "micro"))
print(f1_score(mlb.fit_transform(diagnosis_terms_codes.ICD10.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values),average = "micro"))
print(classification_report(mlb.fit_transform(diagnosis_terms_codes.ICD10.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values),target_names=mlb.classes_))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['A047', 'A419', 'B182', 'D62', 'D631', 'D649', 'D696', 'E039', 'E0842', 'E0942', 'E1042', 'E1142', 'E1342', 'E46', 'E669', 'E780', 'E860', 'E861', 'E870', 'E871', 'E872', 'E875', 'E876', 'F05', 'F17200', 'F329', 'F419', 'G4733', 'I120', 'I129', 'I200', 'I214', 'I252', 'I272', 'I2789', 'I340', 'I348', 'I350', 'I351', 'I352', 'I358', 'I359', 'I425', 'I428', 'I469', 'I472', 'I4892', 'I498', 'I5032', 'I5033', 'I619', 'I739', 'I952', 'I9581', 'I959', 'I97710', 'I97790', 'I9788', 'I9789', 'J189', 'J449', 'J45909', 'J45998', 'J690', 'J918', 'J9811', 'J9819', 'K219', 'K740', 'K7460', 'K7469', 'K922', 'L89139', 'L89149', 'L89159', 'M109', 'M810', 'N170', 'N186', 'N189', 'N390', 'N400', 'P002', 'P220', 'P590', 'R001', 'R0902', 'R569', 'R6520', 'R6521', 'R7881', 'Y832', 'Y838', 'Y848', 'Z23', 'Z3800', 'Z3801', 'Z412', 'Z66', 'Z7901', 'Z794', 'Z853', 'Z8546', 'Z86718', 'Z87891', 'Z950', 'Z95

0.038053708866490336
0.6777420354087735
0.23257039261231657
0.8586997448019077
0.36601044954439277
              precision    recall  f1-score   support

         I10       0.42      0.85      0.56     17287
       I2510       0.28      0.88      0.43     11346
        I509       0.27      0.87      0.41     10907
       I4891       0.27      0.87      0.41     10893
        N179       0.20      0.85      0.32      8104
        E119       0.19      0.86      0.31      7709
        E784       0.21      0.90      0.35      8399
        E785       0.21      0.90      0.35      8399
       J9690       0.14      0.78      0.24      6284
       J9600       0.14      0.78      0.24      6284

   micro avg       0.23      0.86      0.37     95612
   macro avg       0.23      0.85      0.36     95612
weighted avg       0.26      0.86      0.39     95612
 samples avg       0.20      0.67      0.29     95612



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# diagnosis_icd9category=pd.read_csv("/content/drive/My Drive/AlgoIntern/Data/Data2.0/diagnosis_icd9category.csv")
# diagnosis_icd9category['icd9_category']=diagnosis_icd9category.icd9_category.apply(literal_eval)
# diagnosis_icd9category.dropna(subset=['discharge diagnosis:'],inplace=True)
# diagnosis_icd9category.reset_index(drop=True,inplace=True)

In [ ]:
# ! pip install scispacy
# ! pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_core_sci_md-0.2.5.tar.gz

In [ ]:
# import spacy
# import scispacy
# import en_core_sci_md

In [ ]:
# nlp=en_core_sci_md.load()

In [ ]:
# diagnosis_icd9category['medical_terms_list']=diagnosis_icd9category['discharge diagnosis:'].progress_apply(lambda x: nlp(x).ents)
# diagnosis_icd9category['medical_terms_list']=diagnosis_icd9category.medical_terms_list.progress_apply(lambda x: [i.text for a in x for i in a])
# diagnosis_icd9category['medical_terms_list']=diagnosis_icd9category.medical_terms_list.apply(lambda x:list(set([str(i) for i in x])))
# diagnosis_icd9category['med_terms_token']=diagnosis_icd9category.medical_terms_list.progress_apply(lambda x: [word_tokenize(i) for i in x])
# diagnosis_icd9category['med_terms_token']=diagnosis_icd9category.med_terms_token.apply(lambda x:list(set([a for b in x for a in b if a not in list(string.punctuation)])))

In [ ]:
# diagnosis_icd9category.to_csv("/content/drive/My Drive/AlgoIntern/Data/Data2.0/diagnosis_icd9category_tokens_scimd.csv",index=False)

In [ ]:
diagnosis_icd9category=pd.read_csv("/content/drive/My Drive/AlgoIntern/Data/Data2.0/diagnosis_icd9category_tokens_scimd.csv")

In [ ]:
diagnosis_icd9category['icd9_category']=diagnosis_icd9category.icd9_category.apply(literal_eval)
diagnosis_icd9category['med_terms_token']=diagnosis_icd9category.med_terms_token.apply(literal_eval)

In [ ]:
diagnosis_icd9category

In [ ]:
category_med_terms={}
common_diagnosis_terms=["monday","telephone","of","phone",'diagnosis', 'discharge', 'condition', 'followup', 'your', 'instructions','weeks', 'medications', 'appointment', 'hospital', 'care', 'md','admitted', 'office','primary', 'follow-up', 'day', 'appointments', 'prescribed', 'coherent', 'week', 'name', 'ambulatory', 'home', '**location', 'daily','days', 'increased', 'clinic', 'changes','medication','physician','no','doctor','patient','parking','scheduled','schedule','question','treated','independent','month','morning','garage']
for i in frequent_icd9category:
    diagnosis_x=diagnosis_icd9category.loc[diagnosis_icd9category.icd9_category.apply(lambda x: i in x)].copy()
    y=diagnosis_x.med_terms_token.to_list()    
    z=[list(set(a)) for a in y]
    z1=[a for b in z for a in b]
    dict_med=dict(Counter(z1))
    dict_med=dict(sorted(dict_med.items(),key= lambda x: x[1],reverse=True))
    dict_med_list=list(dict_med)
    dict_med_list1=[a for a in dict_med_list if a not in common_diagnosis_terms]
    category_med_terms[i]=dict_med_list1[:30]

In [ ]:
diagnosis_icdcategory_medterms={'icd9category':[i for i in category_med_terms],
                                'frequent_medterms':[category_med_terms[i] for i in category_med_terms]}
diagnosis_icdcategory_medterms_df=pd.DataFrame.from_dict(diagnosis_icdcategory_medterms)

In [ ]:
diagnosis_icdcategory_medterms_df

In [ ]:
y=diagnosis_icdcategory_medterms_df.frequent_medterms.to_list()
y1=[a for b in y for a in b]
medical=dict(Counter(y1))
print([k for (k,v) in medical.items() if v>=16])

In [ ]:
bagofwords=list(set([a for b in diagnosis_icdcategory_medterms_df.frequent_medterms.to_list() for a in b]))

In [ ]:
len(bagofwords)

In [ ]:
set(bagofwords).issubset(model.vocab)

In [ ]:
simi_matrix=np.zeros((len(bagofwords),len(bagofwords)))

In [ ]:
for i in tqdm(range(len(bagofwords))):
  for j in range(len(bagofwords)):
    simi_matrix[i][j]=model.similarity(bagofwords[i],bagofwords[j])

In [ ]:
simi_matrix

In [ ]:
features={}
for i in range(len(bagofwords)-1):
  similar=[]
  for j in range(i+1,len(bagofwords)):
    if simi_matrix[i][j]>=0.7:
      similar.append(bagofwords[j])
  features[bagofwords[i]]=similar
features[bagofwords[-1]]=[]    


In [ ]:
features_list=[features[k]+[k] for k in list(features)]

In [ ]:
len(features_list)

In [ ]:
condensed_feature = list(merge_common(features_list))

In [ ]:
condensed_feature

In [ ]:
len(condensed_feature)

In [ ]:
condensed_feature_dict={}
count=0
for i in condensed_feature:
  condensed_feature_dict['f'+str(count)]=i
  count+=1

In [ ]:
print(condensed_feature_dict)

In [127]:
all_vocab=[a for b in condensed_feature for a in b]

In [128]:
diagnosis_icd9category['med_terms_condensed']=diagnosis_icd9category.med_terms_token.progress_apply(lambda x:list(set(x).intersection(set(all_vocab))))

In [129]:
diagnosis_icd9category

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list,med_terms_token,med_terms_condensed
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","[496, 553, 518, 453, 519, 276]","['discharge diagnosis', 'coronary artery disease/atypical angina', 'hypothyroidism', 'rca', 'cerebral vascular accident', 'questions', 'hypertension', 'discharge condition', 'productive cough', 'concerns', 'chest pain', 'hypercholesterolemia', 'discharge', 'hiatal hernia', 'rib fracture', 's/p motor vehicle', 'office', 'telephone']","[fracture, pain, rib, s/p, discharge, artery, accident, diagnosis, rca, hernia, productive, concerns, cerebral, hiatal, vehicle, telephone, questions, hypothyroidism, hypertension, angina, condition, disease/atypical, office, chest, coronary, hypercholesterolemia, vascular, motor, cough]","[s/p, artery, questions, chest, pain, coronary]"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parki

In [130]:
def return_feature(x):
  feature=[]
  for i in x:
    for j in list(condensed_feature_dict):
      if i in condensed_feature_dict[j]:
        feature.append(j)
  return list(set(feature))


In [131]:
diagnosis_icd9category['features']=diagnosis_icd9category.med_terms_condensed.progress_apply(lambda x:return_feature(x))

In [132]:
diagnosis_icdcategory_medterms_df['feature']=diagnosis_icdcategory_medterms_df.frequent_medterms.progress_apply(lambda x:return_feature(x))

In [133]:
def predict_icd(x):
  category_weights={}
  if len(x):
    for item1 in frequent_icd9category:
      category_features=[a for b in diagnosis_icdcategory_medterms_df.loc[diagnosis_icdcategory_medterms_df.icd9category==item1].feature.to_list() for a in b]
      count=len(list(set(x).intersection(set(category_features))))
      category_weights[item1]=count/len(x)/len(category_features)
  else:
    for item1 in frequent_icd9category:
      category_weights[item1]=0.0

      
  return category_weights
  

In [134]:
diagnosis_icd9category['predicted_icd_weigths']=diagnosis_icd9category.features.progress_apply(lambda x:predict_icd(x))

In [ ]:
diagnosis_icd9category

In [ ]:
diagnosis_icd9category['predicted_icd']=diagnosis_icd9category.predicted_icd_weigths.progress_apply(predictbasedonwts)

In [ ]:
result_predict={'category':[],'precision':[],'recall':[],'f1':[]}
for i in frequent_icd9category:
  actual_p=diagnosis_icd9category.loc[diagnosis_icd9category.icd9_category.apply(lambda x:i in x)].HADM_ID.count()
  tp=diagnosis_icd9category.loc[(diagnosis_icd9category.icd9_category.apply(lambda x:i in x))&(diagnosis_icd9category.predicted_icd.apply(lambda x:i in x))].HADM_ID.count()
  predict_p=diagnosis_icd9category.loc[diagnosis_icd9category.predicted_icd.apply(lambda x:i in x)].HADM_ID.count()
  precision=tp/predict_p
  recall=tp/actual_p
  f1=2*precision*recall/(precision+recall)
  result_predict['category'].append(i)
  result_predict['precision'].append(precision)
  result_predict['recall'].append(recall)
  result_predict['f1'].append(f1)

In [ ]:
pd.DataFrame.from_dict(result_predict)

In [ ]:
total_predicted=len([a for b in diagnosis_icd9category.predicted_icd.to_list() for a in b])
print(total_predicted)
actual=len([a for b in diagnosis_icd9category.icd9_category.apply(lambda x : [i for i in x if i in frequent_icd9category]).to_list() for a in b])
print(actual)
actual_predicted=0
for i in frequent_icd9category:
  actual_predicted+=diagnosis_icd9category.loc[(diagnosis_icd9category.icd9_category.apply(lambda x:i in x))&(diagnosis_icd9category.predicted_icd.apply(lambda x:i in x))].HADM_ID.count()
print(actual_predicted)
recall=actual_predicted/actual
presicion=actual_predicted/total_predicted
print(recall)
print(presicion)
print(2*presicion*recall/(presicion+recall))